In [1]:
import requests
import pandas as pd

response = requests.get(
    "https://api.dexscreener.com/token-boosts/latest/v1",
    headers={"Accept": "*/*"},
)

data = response.json()

# If data is a list, use it directly
if isinstance(data, list):
    tokens = data
elif isinstance(data, dict):
    tokens = data.get("pairs", []) or data.get("data", [])
else:
    tokens = []

# Filter for Solana chain (chainId == "solana")
solana_tokens = [t for t in tokens if t.get("chainId", "").lower() == "solana"]

# Convert to DataFrame
df = pd.DataFrame(solana_tokens)
print(df.head())

                                                 url chainId  \
0  https://dexscreener.com/solana/gzerqy4bbx9ykjv...  solana   
1  https://dexscreener.com/solana/9tbjgexnhsleknc...  solana   
2  https://dexscreener.com/solana/djabfe6ejhvzs4r...  solana   
3  https://dexscreener.com/solana/dcmeu6dadnqmnam...  solana   
4  https://dexscreener.com/solana/6hea81v3sf6cbfw...  solana   

                                   tokenAddress  \
0  GZerqy4BBx9YkjVsyuynkb31Pb4YvzPqpJAw8P9Bpump   
1  9tbJGeXNHsLEkncTRSUF7Cn2k6Meji5wvz4jpagmpump   
2  DjAbfe6EjhvZs4RAajYs8r3jcC1cFqqK4beTfGS7Lidd   
3   DCmeU6DAdNqMNamxw4Uw4Zp6rdVwwRcJDDpbPVJCULT   
4  6HEa81V3sf6CBFWCPSVYN88J44fnzTxgrVrxzHDJpump   

                                         description  \
0  Born in cosmic wilds, now cruising galaxies wi...   
1  Solana Agent Middleware - An AI-powered framew...   
2  I know that I'm supposed to add a catchy sales...   
3  COLORS is a community memecoin that’s growing ...   
4  The Dinging $BELL on SOL.

In [2]:
import requests
import os
from dotenv import load_dotenv
import pandas as pd
from joblib import dump

# Load .env
load_dotenv()
Birdeye_Api_key = os.getenv("Birdeye_Api_key")

url = "https://public-api.birdeye.so/defi/v2/tokens/new_listing"

headers = {
    "accept": "application/json",
    "x-chain": "solana",
    "x-api-key": Birdeye_Api_key
}

all_tokens = []
batch_size = 20
max_records = 100

for offset in range(0, max_records, batch_size):
    params = {
        "limit": batch_size,
        "offset": offset,
        "meme_platform_enabled": "false"
    }
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    
    tokens = data.get("data", {}).get("items", [])
    if not tokens:
        print(f"⚠️ No items at offset {offset}, stopping early.")
        break
    all_tokens.extend(tokens)

# Convert to pandas DataFrame
df = pd.DataFrame(all_tokens)

print(f"✅ Total tokens fetched: {len(df)}")
print("\nDataFrame Preview:")
print(df.head())

# Save to joblib
if not df.empty:
    dump(df, "birdeye_new_listings.pkl")
    print("\n💾 Data saved to birdeye_new_listings.pkl")
else:
    print("\n⚠️ No tokens fetched, nothing saved.")


✅ Total tokens fetched: 100

DataFrame Preview:
                                        address    symbol  \
0  8W7t1RrynK6qAR5dAH96zXaQzLtAiF1XjMwHsT1Sbonk    THREAT   
1  4Ho9m7Ln521vShcwNBF6M4XHZwoFWFW6bZNnp4rMsEaj  Mask一eth   
2   qUxv9w4oegLrYCpKBoBjAFasgVDyiZuSx8RvnuCbonk       LLF   
3  6jDhdJ7mn7RtFhEu4c6KK23hLoeCbaSQ3EzCaRREZhqD   TeslaAI   
4  72PCBA7YYYgsVm5Mf1vBPBdMzLNGNMZhRhEGy2FMmRp3    asdzxv   

                  name  decimals                         source  \
0         Large Threat         6              raydium_launchlab   
1        MetaMask Coin         6  meteora_dynamic_bonding_curve   
2  Large Latino Forces         6              raydium_launchlab   
3             Tesla AI         6                       fluxbeam   
4               asdzxv         6                     raydium_cp   

      liquidityAddedAt                                            logoURI  \
0  2025-09-22T15:45:28  https://thumbnails.padre.gg/SOLANA-D6jceX98RoN...   
1  2025-09-22T15:45:27  http

In [3]:

import os
from dotenv import load_dotenv
import requests
import pandas as pd
import joblib

# Load .env
load_dotenv()
API_KEY = os.getenv("MORALIS_API_KEY")

if not API_KEY:
    raise ValueError("MORALIS_API_KEY not found in .env")

url = "https://solana-gateway.moralis.io/token/mainnet/4zLj7pCr1kQSRPNReQAqbpwvtEdKrkwj2peHYTyB92sS/top-holders"
params = {"limit": 100}

headers = {
    "Accept": "application/json",
    "X-API-Key": API_KEY
}

response = requests.get(url, headers=headers, params=params)

print("HTTP", response.status_code)

if response.status_code == 200:
    data = response.json()
    holders = data.get("result", [])  # Moralis usually puts the list in "result"

    # Convert to DataFrame
    df = pd.DataFrame(holders)

    # Save with joblib
    joblib.dump(df, "top_holders.pkl")

    print("✅ Data saved to top_holders.pkl")
    print(df.head())
else:
    print(response.text)


HTTP 200
✅ Data saved to top_holders.pkl
           balance  balanceFormatted  isContract  \
0  777844972352764  777844972.352764       False   
1   34116861428818   34116861.428818       False   
2   31882426573161   31882426.573161       False   
3   23019521397142   23019521.397142       False   
4   15355790657762   15355790.657762       False   

                                   ownerAddress              usdValue  \
0  Cn7fEU6Q8rGqQSEBVyPX2XGofvMdbWAsNn2TuUo3oiSS  5418.468077409354024   
1  2HyCReJKEchRa6VZhuLRtchGCtTzn5s9NTMk6Tcn3LGL   237.658056713146188   
2  DWxWLNHWN42q8UT6ypbhh4j1wCKTjnnvM2WDCT7DYJ3d   222.092983508639526   
3  GsJYPEiewEm5QKMmagiu2poUk7BgYoiquqPHF3Bm9pm3   160.353986052491172   
4  A1JQZ8wX4mj252CcB7KRpK88t1usuwbLrpukR2pbXGCC   106.968437721970092   

   percentageRelativeToTotalSupply  
0                            77.80  
1                             3.41  
2                             3.19  
3                             2.30  
4                     

In [4]:
import os
from dotenv import load_dotenv
import requests
import pandas as pd
import joblib

# Load API key
load_dotenv()
API_KEY = os.getenv("MORALIS_API_KEY")

if not API_KEY:
    raise ValueError("MORALIS_API_KEY not found in .env")

def fetch_wallet_tokens(wallet_address: str):
    url = f"https://solana-gateway.moralis.io/account/mainnet/{wallet_address}/tokens"

    headers = {
        "Accept": "application/json",
        "X-API-Key": API_KEY
    }

    response = requests.get(url, headers=headers)

    print("HTTP", response.status_code)

    if response.status_code == 200:
        data = response.json()

        # API returns a list
        tokens = data if isinstance(data, list) else data.get("result", [])

        if not tokens:
            print("⚠️ No tokens found in this wallet.")
            return pd.DataFrame()

        df = pd.DataFrame(tokens)

        # ✅ Keep only the selected fields
        cols_to_keep = ["associatedTokenAddress", "amount", "name", "symbol"]
        df = df[[col for col in cols_to_keep if col in df.columns]]

        # Save with joblib
        joblib.dump(df, "wallet_tokens.pkl")

        print("✅ Data saved to wallet_tokens.pkl")
        print(df.head())
        return df
    else:
        print("❌ Error:", response.text)
        return pd.DataFrame()


if __name__ == "__main__":
    wallet_address = input("🔑 Enter Solana wallet address: ").strip()
    df = fetch_wallet_tokens(wallet_address)


HTTP 404
❌ Error: {"message":"Resource not found. Please check out the API documentation: https://docs.moralis.com/"}


In [ ]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv

def fetch_wallet_tokens(wallet_address: str):
    load_dotenv()
    API_KEY = os.getenv("MORALIS_API_KEY")
    if not API_KEY:
        raise ValueError("MORALIS_API_KEY not found in .env")
    url = f"https://solana-gateway.moralis.io/account/mainnet/{wallet_address}/tokens"
    headers = {
        "Accept": "application/json",
        "X-API-Key": API_KEY
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        tokens = data if isinstance(data, list) else data.get("result", [])
        if not tokens:
            return pd.DataFrame()
        df = pd.DataFrame(tokens)
        cols_to_keep = ["associatedTokenAddress", "amount", "name", "symbol"]
        df = df[[col for col in cols_to_keep if col in df.columns]]
        return df
    else:
        return pd.DataFrame()